## Scraping Yelp Data

### 1. What the Scraper is Doing

The script performs the following tasks:

- **Input and Output CSVs**: It reads URLs from an input CSV file and writes the extracted data to an output CSV file.
- **Fetching Web Page Content**: For each URL, it sends an HTTP GET request, simulating a browser to avoid being blocked.
- **Parsing HTML**: It uses BeautifulSoup to parse the HTML content and searches for specific tags (`span` with class `y-css-kw85nd`).
- **Extracting Data**: It retrieves text within anchor (`<a>`) tags nested inside the targeted `span` elements, limited to 5 words per URL.
- **Handling Errors**: The script includes error handling for failed requests, ensuring that the script doesn't crash if a URL is unreachable.
- **Writing to CSV**: The extracted words are written to specific columns (`Label 1` to `Label 5`) in the output CSV.




---




### 2. How to Inspect a Website and Find the Information to Extract

When you want to scrape data from a website, you first need to understand how the information is structured in the HTML. Here's a step-by-step guide on how to do this:

##### 1. Visual Inspection:

- **Start by Visiting the Webpage**: Open the webpage in a browser (e.g., Chrome, Firefox).
- **Identify the Data You Need**: Look at the page and decide what specific information you want to extract (e.g., product names, prices, reviews, etc.).

##### 2. Inspecting the HTML Code:

- **Right-Click on the Desired Data**: Find the piece of data you want on the page (e.g., the name of a restaurant). Right-click on it and select "Inspect" or "Inspect Element" from the context menu.

Note: Make sure that the HTML code that you are viewing is for the web browser and not the mobile version. You can change the version by clicking on the dimension (right above the screen once the HTML code is opened) and selecting the right version. 


- **Examine the HTML Panel**: This opens the developer tools in your browser, highlighting the HTML element that corresponds to the data you selected.
  - **Understanding Tags and Attributes**: HTML is made up of various tags like `<div>`, `<span>`, `<a>`, etc. Each tag can have attributes like `class`, `id`, `href`, etc., that help identify and style it.

##### 3. Finding Patterns:

- **Look for Classes or IDs**: Check if the element has a unique class or id. These attributes are usually consistent across similar elements on the page (e.g., all product names might be in `<span>` tags with a class `product-title`).
- **Navigate Up or Down the HTML Structure**: Sometimes, the data is nested inside another tag. You might need to go one level up or down in the HTML hierarchy to find a container that groups several elements you’re interested in.
- **Identify Repeating Patterns**: If you’re scraping multiple items (like a list of products), find the common pattern in the HTML structure that repeats for each item.

##### 4. Testing Your Selection:

- **Use the Console for Testing**: In the browser's developer tools, you can use the Console to write small JavaScript snippets to see if your selection correctly identifies all desired elements.
- **Copy the Selector**: Once you’ve identified the correct tag and class or id, you can write the selector in your code to target these elements.

This process helps you translate what you see on the webpage into code that can reliably extract the information.


In [1]:
# Incomplete Scraper Script for Students

import csv
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.exceptions import RequestException
import time


In [8]:
# Function to extract specific words from a given URL
def extract_words_from_url(url):
    try:
# Create a session object to persist settings across requests
        session = requests.Session()
    
        # TODO: Add retry logic to handle server errors (Hint: Use the Retry and HTTPAdapter classes)
        
        retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retries)
        session.mount("http://", adapter)
        session.mount("https://", adapter)

        
        # TODO: Set up headers to mimic a browser (Hint: User-Agent and Accept headers)
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",}

        # Make the request to the URL (Hint: use session.get)
        response = session.get(url, headers=headers, timeout=10)  # <--- Replace with your request code
        # Make sure to raise an exception for any bad response
        response.raise_for_status()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")  # <--- Replace with BeautifulSoup parsing code
        
        # Initialize an empty list to store extracted words
        words = []
        
        # TODO: Find all <span> elements with a specific class (Hint: use soup.find_all)
        # and extract keywords from <a> tags inside those spans
        for span in soup.find_all("span", class_="y-css-1cafv3i"):
            for a_tag in span.find_all("a"):
                word = a_tag.get_text(strip=True)
                if word:
                    words.append(word)
            
        # Return the first 5 words or fewer if less
        return words[:5]
        
    except RequestException as e:
        print(f"Error scraping {url}: {e}")  # Log the error message
        return [""] * 5  # Return a list of 5 empty strings if an error occurs
        
    finally:
        time.sleep(0)


In [12]:
def scrape_urls_from_csv(input_csv, output_csv):
    # Open the input CSV for reading and the output CSV for writing
    with open(input_csv, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:

        # TODO: Use csv.DictReader to read the CSV file and get a list of dictionaries, 
        # where each row is represented as a dictionary with column headers as keys.
        reader = csv.DictReader(infile)
        # Get the fieldnames (i.e., the column headers from the input CSV)
        fieldnames = reader.fieldnames

        # TODO: Use csv.DictWriter to write rows into the output CSV file, 
        # making sure to include the same fieldnames as the input CSV.

        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        # Write the headers (column names) into the output CSV file
        writer.writeheader()

        # Iterate over each row in the input CSV
        for row in reader:
            # TODO: Extract the URL from the 'restaurant_url' column in the CSV.

            url = row.get("restaurant_url")

            # If the URL is missing, skip the row and print a message
            if not url:
                print(f"Skipping row with missing URL: {row}")
                continue

            # Print the URL being processed for debugging purposes
            print(f"Processing URL: {url}")

            # TODO: Call the extract_words_from_url function to get a list of words
            extracted_words = extract_words_from_url(url)

            # TODO: For each word in the extracted list, assign it to a new column (e.g., 'Label 1', 'Label 2', etc.)
            # Ensure the columns are named 'Label 1', 'Label 2', etc., and each word is assigned to the correct column.
            for i, word in enumerate(extracted_words):
                row[f"Label {i + 1}"] = word
            for i in range(len(extracted_words) + 1, 6): #filling empty columns with empty strings
                row[f"Label {i}"] = ""
                
            # Write the updated row into the output CSV
            writer.writerow(row)

            # TODO: Add a small delay between processing each URL to avoid overwhelming the server
            # Hint: A delay of 1 second could be more appropriate

if __name__ == "__main__":
    # TODO: Specify the input and output CSV file paths
    input_csv = "/Users/kaykaydaou/Desktop/MMA/MMA WINTER 25/W1 TEXT ANALYTICS/Labs/URLs to scrape.csv"
    output_csv = "/Users/kaykaydaou/Desktop/MMA/MMA WINTER 25/W1 TEXT ANALYTICS/Labs/output.csv"
    # Call the main scraping function
    scrape_urls_from_csv(input_csv, output_csv)


Processing URL: https://www.yelp.com/biz/torchys-tacos-fayetteville-2
Processing URL: https://www.yelp.com/biz/charlies-bakery-anchorage
Processing URL: https://www.yelp.com/biz/haute-quarter-grill-anchorage
Processing URL: https://www.yelp.com/biz/club-paris-anchorage
Error scraping https://www.yelp.com/biz/club-paris-anchorage: 403 Client Error: Forbidden for url: https://www.yelp.com/biz/club-paris-anchorage
Processing URL: https://www.yelp.com/biz/f-street-station-anchorage
Error scraping https://www.yelp.com/biz/f-street-station-anchorage: 403 Client Error: Forbidden for url: https://www.yelp.com/biz/f-street-station-anchorage
Processing URL: https://www.yelp.com/biz/great-harvest-bread-anchorage-anchorage
Processing URL: https://www.yelp.com/biz/anchorage-museum-anchorage-2
Error scraping https://www.yelp.com/biz/anchorage-museum-anchorage-2: 403 Client Error: Forbidden for url: https://www.yelp.com/biz/anchorage-museum-anchorage-2
Processing URL: https://www.yelp.com/biz/ginger-

### Key Details of the Code

**Session with Retry Logic**:  
This makes the code more robust by handling intermittent failures when accessing the URLs. It retries the request up to 5 times with increasing delays (exponential backoff).

**Browser-Like Headers**:  
The headers help disguise the script as a browser, preventing some websites from blocking it.

**HTML Parsing with BeautifulSoup**:  
The code searches for specific `<span>` elements with a defined class to locate words/keywords and extracts them.

**CSV Handling**:  
The input CSV is read row by row, URLs are processed, and extracted data is written back into a new CSV with additional columns (`Label 1` to `Label 5`).

**Error Handling**:  
If a request fails, the error is logged, and the script moves to the next URL, ensuring that an error doesn't stop the entire process.

**Delay Between Requests**:  
Although set to `0` for now, the `time.sleep(0)` can be adjusted to introduce a delay between requests to avoid overloading a server or getting IP blocked.
